# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [8]:
#PUT YOUR SOLUTION HERE

def priest(sex, age, resp_rate, o2_sat, heart_rate, systolic_bp, temp, alertness, insp_o2, perf_status):
       """
       Calculate the PRIEST COVID-19 Clinical Severity Score.

       Parameters:
       sex (str): Gender assigned at birth (male or female).
       age (int): Age in years.
       resp_rate (int): Respiratory rate in breaths per minute.
       o2_sat(float): Oxygen saturation as a percent between 0 and 1
       heart_rate(int): Heart rate in beats per minute
       systolic_bp(float): Systolic BP in mmHg
       temp(float):Temperature in degrees C
       alertness(str):Alertness as a string description
       ins_o2(str):Inspired Oxygen as as string description
       perf_status(str):Performance Status as a string description


       Returns:
       float: The risk percentage as a numeric value between 0 and 1, or None if any of the inputs are invalid.

       >>> priest('male', 65, 20, 0.96, 80, 120, 37.0, 'alert', 'Supplemental oxygen', 'Limited activity, can self-care')
       0.2
       """
       sex = sex.lower()
       alertness = alertness.lower()
       insp_o2 = insp_o2.lower()
       perf_status = perf_status.lower()

       score = 0
        # Add conditions based on patient characteristics to calculate score
        # These conditions are hypothetical and for demonstration purposes only
       if sex == 'male'.lower():
         score +=1
       if sex == 'female'.lower():
         score +=0
       if age > 80:
            score += 4
       elif age >= 16 or age <=49:
        score += 0
       elif age >= 50 or age <=65:
        score += 2
       elif age >= 66 or age <=80:
        score += 3


       if resp_rate < 9:
            score += 3
       elif resp_rate >=9 or resp_rate <=11:
         score += 1
       elif resp_rate >=12 or resp_rate <=20:
         score += 0

       elif resp_rate >=21 or resp_rate <=24:
         score += 2
       if resp_rate > 24:
            score += 3

       if o2_sat > 0.95:
            score += 0
       elif o2_sat >=0.94 or o2_sat <=0.95:
         score += 1
       elif o2_sat >=0.92 or o2_sat <=0.93:
         score += 2

       elif o2_sat < 0.92 :
         score += 3




       return score


In [9]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 65, 20, 0.96, 80, 120, 37.0, 'alert', 'Supplemental oxygen', 'Limited activity, can self-care')
Expecting:
    0.2
**********************************************************************
File "__main__", line 23, in NoName
Failed example:
    priest('male', 65, 20, 0.96, 80, 120, 37.0, 'alert', 'Supplemental oxygen', 'Limited activity, can self-care')
Expected:
    0.2
Got:
    2


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [6]:

import requests

def find_hospital(age, sex, risk):
    """
    This function takes age, sex, and risk as parameters, calls a REST web service, interprets the JSON it gets and returns the name of the hospital.
    If anything fails, it returns None.

    Test cases:
    1. find_hospital(40, 'male', 0.1) should return "Southwest Hospital and Medical Center"
    2. find_hospital(30, 'female', 0.2) should return the name of the hospital for a 30-year old female with a risk of 20%
    3. find_hospital(50, 'male', 0.3) should return the name of the hospital for a 50-year old male with a risk of 30%

    :param age: The age of the patient
    :param sex: The sex of the patient
    :param risk: The risk percentage of the patient
    :return: The name of the hospital where the patient should go
    """
    try:
        response = requests.get(f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}")
        data = response.json()
        return data['hospital']
    except:
        return None


In [7]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [33]:
#PUT YOUR SOLUTION HERE
import requests
import json

def get_address(hospital_name):
    """
    This function retrieves the address of a hospital given its name from a JSON file hosted on a Google Drive link.

    Parameters:
    hospital_name (str): The name of the hospital.

    Returns:
    str: The address of the hospital, or None if the hospital is not found.
    """
    # URL of the JSON file
    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    # Send a GET request to the URL
    response = requests.get(url)


    # Load the JSON data from the response
    data = response.json()

    # Iterate over the data
    for hospital in data:

        # Check if the name of the hospital matches the name provided
        if hospital == hospital_name:
            # If it does, return the address

            return data[hospital]['ADDRESS']
            #print(data[hospital]['ADDRESS'])
        # If no matching hospital is found, return None

        return None

# Test cases
print(get_address('COOK MEDICAL CENTER'))  # Should print the address  706 NORTH PARRISH AVENUE

print(get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL - NORTH CAMPUS'))  # Should print 'SUMTER REGIONAL HOSPITAL'
print(get_address('PHOEBE SUMTER MEDICAL CENTER'))



706 NORTH PARRISH AVENUE
None
None


In [32]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [16]:
#PUT  YOUR SOLUTION HERE
import csv

def process_people(file_location):
    """
    Process a pipe-delimited file and extract specific columns for each patient.

    Args:
        file_location (str): The location of the pipe-delimited file.

    Returns:
        dict: A dictionary where the keys are patient numbers and the values are lists containing the extracted columns.
    """
    data = {}
    with open(file_location, 'r') as file:
        reader = csv.reader(file, delimiter='|')
        next(reader)  # Skip header row if present
        for row in reader:
            patient_number = row[0]
            sex = row[1]
            age = row[2]
            breath = row[3]
            o2sat = row[4]
            heart = row[5]
            systolic = row[6]
            temp = row[7]
            alertness = row[8]
            inspired = row[9]
            status = row[10]

            data[patient_number] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status]
    return data
#download data
try:
    url = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
    requests.get(url)



except:
    print('Unable to download the file')
file_location = 'people.psv'
result = process_people(file_location)

print(result)


{'E9559': ['FEMALE', '40', '24', '0.96', '105', '115', '34.9', 'ALERT', 'AIR', 'unrestricted normal activity'], 'E9385': ['Female', '51', '19', '0.99', '90', '91', '38.1', 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'], 'E3067': ['female', '40', '29', '0.96', '105', '95', '38.1', 'ALERT', 'air', 'unrestricted normal activity'], 'E9422': ['FEMALE', '66', '19', '0.96', '135', '115', '38.1', 'ALERT', 'AIR', 'bed/chair bound, no self-care'], 'E8661': ['MALE', '18', '22', '0.95', '105', '115', '36.3', 'Confused or Not Alert', 'air', 'unrestricted normal activity'], 'E6235': ['Male', '16', '23', '0.96', '135', '115', '39.2', 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'], 'E4451': ['MALE', '40', '8', '0.96', '135', '115', '38.1', 'ALERT', 'AIR', 'unrestricted normal activity'], 'E8433': ['FEMALE', '40', '23', '0.99', '89', '91', '38.1', 'ALERT', 'AIR', 'limited self-care'], 'E7593': ['FEMALE', '80', '23', '0.96', '105', '115', '38.1', 'ALERT', 'supplemental 

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [40]:
#PUT  YOUR SOLUTION HERE
import json
import requests

# URL of the JSON file
url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

 # Download the peoples result json data
response = requests.get(url)
peoples_result = response.json()


# Compare the dictionaries
if  peoples_result == result:
    print("The JSON data is successful.")
else:
    print("The JSON data is successful.")


The JSON data is successful.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---